# 1. Get the SNPs at positions of interest from both early pool (pool1) and late pool (pool4)

In [12]:
# Code improved with ChatGPT

# Define the chromosome and position range you want to filter by
desired_chr = ['A01','A02','A03']  # Change this to the chromosome you're interested in (e.g., 'chr1', 'chr2', etc.)
min_position = 1  # Minimum position in the range (inclusive)
max_position = 10000000  # Maximum position in the range (inclusive)

# Open the input VCF file and an output file for the filtered results
with open('pool1/pool1.ann.vcf', 'r') as infile, open('pool1/pool1_filtered.vcf', 'w') as outfile:
    for line in infile:
        # Skip header lines (those that start with '#')
        if line.startswith('#'):
            outfile.write(line)
        else:
            # Split the line by tab and check the chromosome and position (columns 1 and 2)
            columns = line.strip().split('\t')
            chr_column = columns[0]  # Chromosome
            position_column = int(columns[1])  # Position (converted to integer)
            
            # Filter based on chromosome and position range
            if chr_column in desired_chr and min_position <= position_column <= max_position:
                outfile.write(line)

#print(f"Filtered VCF file written to 'filtered_output.vcf' with {desired_chr} and positions between {min_position} and {max_position}")

# Repeat for pool4
with open('pool4/pool4.ann.vcf', 'r') as infile, open('pool4/pool4_filtered.vcf', 'w') as outfile:
    for line in infile:
        # Skip header lines (those that start with '#')
        if line.startswith('#'):
            outfile.write(line)
        else:
            # Split the line by tab and check the chromosome and position (columns 1 and 2)
            columns = line.strip().split('\t')
            chr_column = columns[0]  # Chromosome
            position_column = int(columns[1])  # Position (converted to integer)
            
            # Filter based on chromosome and position range
            if chr_column in desired_chr and min_position <= position_column <= max_position:
                outfile.write(line)


# 2. Get the intersecting SNP positions from pool1_chra01.vcf and pool4_chra01.vcf

In [11]:
# Example usage
vcf_file1 = 'pool1/pool1_filtered.vcf'  # Replace with the path to your first VCF file
vcf_file2 = 'pool4/pool4_filtered.vcf'  # Replace with the path to your second VCF file
output_file = 'pool1_pool4_intersect_snps.vcf'  # The name of the output file containing the intersection


# Function to extract positions from a VCF file
def get_positions(vcf_file):
    positions = set() # store entries in a set as it will only output unique entries
    with open(vcf_file, 'r') as file:
        for line in file:
            if line.startswith('#'):
                continue  # Skip header lines
            columns = line.strip().split('\t')
            position_column = int(columns[1])  # Position is in the 2nd column
            positions.add(position_column)
    return positions

# Function to find the intersection of positions between two VCF files
def find_position_intersection(vcf_file1, vcf_file2, output_file):
    # Get positions from both VCF files
    positions_1 = get_positions(vcf_file1)
    positions_2 = get_positions(vcf_file2)
    
    # Find the common positions (intersection)
    common_positions = positions_1.intersection(positions_2)

    # Open the output file to write the intersecting positions
    with open(output_file, 'w') as outfile:
        with open(vcf_file1, 'r') as file1:
            for line in file1:
                if line.startswith('#'):
                    outfile.write(line)  # Write header lines directly to output
                else:
                    columns = line.strip().split('\t')
                    position_column = int(columns[1])  # Position
                    
                    # Check if the position is in the intersection
                    if position_column in common_positions:
                        outfile.write(line)

    print(f"Intersecting positions written to '{output_file}'")


find_position_intersection(vcf_file1, vcf_file2, output_file)



Intersecting positions written to 'pool1_pool4_intersect_snps.vcf'


# 3. Filter intersecting SNPs for those with high impact (column 8, Annotation_Impact, Gene_Name)

In [6]:
impact_level="HIGH"

with open('pool1_pool4_intersect_snps.vcf', 'r') as infile, open('bothpools_highimpact_snps.vcf', 'w') as outfile:
    for line in infile:
        if line.startswith('#'):
            outfile.write(line)
        else:
            columns = line.strip().split('\t')
            ann_column = columns[7]
            
            ann_fields = ann_column.strip().split('|')
            annotation_impact = ann_fields[2]

            if annotation_impact == impact_level:
                outfile.write(line)



In [8]:
# Get set of gene names
vcf_file = 'bothpools_highimpact_snps.vcf'

gene_name_set = set()

with open(vcf_file, 'r') as infile:
    for line in infile:
        if line.startswith('#'):
            continue
        else:
            columns = line.strip().split('\t')
            info = columns[7]
            fields = info.strip().split('|')
            gene_name = fields[3]

            gene_name_set.add(gene_name)

# Open the input GFF3 file and the output file where the filtered data will be saved
#input_gff_file = '/Users/magilin/local_analysis/resources-all/genomes/b_rapa/ro18_release59/Brassica_rapa_ro18.SCU_BraROA_2.3.60.primary_assembly.A01.gff3'  # Replace with your GFF3 file path
input_gff_file = '/Users/magilin/local_analysis/resources-all/genomes/b_rapa/ro18_release60/Brassica_rapa_ro18.SCU_BraROA_2.3.60.primary_assembly.A02.gff3'
output_gff_file = 'filtered_A02_highimpact_genes.gff3'  # Path to save filtered GFF3 file

with open(input_gff_file, 'r') as infile, open(output_gff_file, 'w') as outfile:
    outfile.write('# These are genes filtered for high impact SNPs from the early flowering pool.')
    
    for line in infile:
        if line.startswith('###'):
            continue
        elif line.startswith('#'):
            # If the line is a comment (starts with '#'), write it to the output without modification
            outfile.write(line)
        else:
            # Split the line by tabs (GFF3 format is tab-delimited)
            columns = line.strip().split('\t')
            # Extract the gene name from the 'Attributes' column (GFF3 usually stores gene names here)
            attributes = columns[8]
            
            # Extract the gene ID (this assumes the format 'ID=GeneName' in the Attributes field)
            gene_name = None
            for attribute in attributes.split(';'):
                if attribute.startswith('ID='):
                    gene_name = attribute.split(':')[1]
                    break
            
            # Check if the gene is in the set of interest
            if gene_name in gene_name_set:
                # If it is, write the line to the output file
                outfile.write(line)


# 4. Do for moderate and modifier impact



In [8]:
impact_level="MODIFIER"

with open('pool1_pool4_intersect_snps.vcf', 'r') as infile, open('bothpools_modifierimpact_snps.vcf', 'w') as outfile:
    for line in infile:
        if line.startswith('#'):
            outfile.write(line)
        else:
            columns = line.strip().split('\t')
            ann_column = columns[7]
            
            ann_fields = ann_column.strip().split('|')
            annotation_impact = ann_fields[2]

            if annotation_impact == impact_level:
                outfile.write(line)

with open('bothpools_modifierimpact_snps.vcf', 'r') as infile, open('modifierimpact_snps.bed', 'w') as outfile:
    for line in infile:
        if line.startswith('#'):
            continue
        else:
            columns = line.strip().split('\t')
            chromosome= columns[0]
            start = int(columns[1])-1
            end = int(columns[1])

            outfile.write('\t'.join([chromosome, str(start), str(end)]) + '\n')

vcf_file = 'bothpools_modifierimpact_snps.vcf'

gene_name_set = set()

with open(vcf_file, 'r') as infile:
    for line in infile:
        if line.startswith('#'):
            continue
        else:
            columns = line.strip().split('\t')
            info = columns[7]
            fields = info.strip().split('|')
            gene_name = fields[3]

            gene_name_set.add(gene_name)


# Get set of gene names
vcf_file = 'bothpools_modifierimpact_snps.vcf'

gene_name_set = set()

with open(vcf_file, 'r') as infile:
    for line in infile:
        if line.startswith('#'):
            continue
        else:
            columns = line.strip().split('\t')
            info = columns[7]
            fields = info.strip().split('|')
            gene_name = fields[3]

            gene_name_set.add(gene_name)

# Open the input GFF3 file and the output file where the filtered data will be saved
input_gff_file = '/Users/magilin/local_analysis/resources-all/genomes/b_rapa/ro18_release59/Brassica_rapa_ro18.SCU_BraROA_2.3.60.primary_assembly.A01.gff3'  # Replace with your GFF3 file path
output_gff_file = 'filtered_modifierimpact_genes.gff3'  # Path to save filtered GFF3 file

with open(input_gff_file, 'r') as infile, open(output_gff_file, 'w') as outfile:
    outfile.write('# These are genes filtered for high impact SNPs from the early flowering pool.')
    
    for line in infile:
        if line.startswith('###'):
            continue
        elif line.startswith('#'):
            # If the line is a comment (starts with '#'), write it to the output without modification
            outfile.write(line)
        else:
            # Split the line by tabs (GFF3 format is tab-delimited)
            columns = line.strip().split('\t')
            # Extract the gene name from the 'Attributes' column (GFF3 usually stores gene names here)
            attributes = columns[8]
            
            # Extract the gene ID (this assumes the format 'ID=GeneName' in the Attributes field)
            gene_name = None
            for attribute in attributes.split(';'):
                if attribute.startswith('ID='):
                    gene_name = attribute.split(':')[1]
                    break
            
            # Check if the gene is in the set of interest
            if gene_name in gene_name_set:
                # If it is, write the line to the output file
                outfile.write(line)
